In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain.llms import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI

In [37]:
class HistoryQuiz:

    def create_history_question(self, topic):
        """
        This method should output a historical question about the topic that
        has a date as a correct answer. For example:

            On what date did World War 2 end?
        """
        system_template = "You write single quiz question about {topic}. You only return the quiz question"
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

        human_template = "{question}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

        ques = "Give me a quiz question where the correct answer is a specific date."
        request = chat_prompt.format_prompt(topic=topic, question=ques).to_messages()

        chat = ChatOpenAI()
        result = chat(request)
        
        return result.content


    def get_AI_answer(self, question):
        """
        This method should get the answer to the historical question from the method above.
        The answer should be datetime format

        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        """
        output_parser = DatetimeOutputParser()
        system_template = "You answer quiz questions with just a date."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

        human_template ="""Answer the user's question:

        {question}

        {format_instructions}"""
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

        format_instructions = output_parser.get_format_instructions()
        request = chat_prompt.format_prompt(question=question,
                                            format_instructions=format_instructions).to_messages()

        chat = ChatOpenAI()
        result = chat(request)

        correct_datetime = output_parser.parse(result.content)

        return correct_datetime


    def get_user_answer(self, question):
        """
        This method should grab a user answer and convert it to a datetime format
        """
        print(question)
        print("\n")

        year = int(input("Enter the year: "))
        month = int(input("Enter the month (1-12): "))
        day = int(input("Enter the day (1-31): "))
        user_datetime = datetime(year, month, day)

        return user_datetime

    
    def check_user_answer(self, user_answer, ai_answer):
        """
        Should check the user answer against the AI answer and return the difference.
        """
        difference = user_answer - ai_answer
        formatted_difference = str(difference)
        print("The difference between the user and ai answer is:", formatted_difference)

In [42]:
quiz_bot = HistoryQuiz()

In [43]:
question = quiz_bot.create_history_question(topic="World War 2")

In [44]:
question

'On what date did the Japanese attack Pearl Harbor, drawing the United States into World War II?'

In [45]:
ai_answer = quiz_bot.get_AI_answer(question)

In [46]:
ai_answer

datetime.datetime(1941, 12, 7, 0, 0)

In [47]:
user_answer = quiz_bot.get_user_answer(question)

On what date did the Japanese attack Pearl Harbor, drawing the United States into World War II?




Enter the year:  1942
Enter the month (1-12):  12
Enter the day (1-31):  7


In [48]:
user_answer

datetime.datetime(1942, 12, 7, 0, 0)

In [49]:
quiz_bot.check_user_answer(user_answer, ai_answer)

The difference between the user and ai answer is: 365 days, 0:00:00
